<hr style="border:10px solid gray">

# Simple running example of DAHSI

<hr style="border:2px solid gray">

⚠️ In order to run this notebook, you need to have installed all the dependancies needed to run DAHSI as listed in the $\texttt{README.md}$ file in this repository.

<hr style="border:2px solid gray">

⚠️ This notebook is for illustration of the method purposes only and so the toy problem chosen contains no hidden variables to be able to go through the code in a few minutes. This notebook will give you the tools and understanding necessary to be able to build your own problem and solve it using DAHSI.

<hr style="border:2px solid gray">

In this notebook we will show a simple running example of the DAHSI algorithm presented in the paper ... 

 

## Generating the data

We will work with the Lorenz system as it is a classical example of chaotic systems.

The data we will use for showing a simple running example will be comprised of $N = 501$ time points, with $\Delta t = 0.01$ and noise ...

We will save the time-series for each variable in $\texttt{.dat}$ files.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

def RK4(f, t, y0, args=()):
    n = len(t)
    y = np.zeros((n, len(y0)))
    y[0] = y0
    for i in range(n - 1):
        h = t[i+1] - t[i]
        k1 = f(t[i], y[i], *args)
        k2 = f(t[i] + h / 2., y[i] + k1 * h / 2., *args)
        k3 = f(t[i] + h / 2., y[i] + k2 * h / 2., *args)
        k4 = f(t[i] + h, y[i] + k3 * h, *args)
        y[i+1] = y[i] + (h / 6.) * (k1 + 2*k2 + 2*k3 + k4)
    return y

def lorenz_K(t, y, sigma, rho, beta):
    return np.array([sigma * (y[1] - y[0]), 
                     y[0]*(rho - y[2]) - y[1], 
                     y[0]* y[1] - beta*y[2]])

# Generate data
y0 = [-8.0, 7.0, 27.0]
sigma = 10.0
rho = 28.0
beta = 8.0/3

dt = 0.01
N = 500+1
tfin = dt * (N - 1)
t = np.linspace(0, tfin, N)

sol = RK4(lorenz_K, t, y0, args=(sigma,rho,beta))

x = sol[:,0]
y = sol[:,1]
z = sol[:,2]

xfile = open("datax_Lorenz.dat", "w")
yfile = open("datay_Lorenz.dat", "w")
zfile = open("dataz_Lorenz.dat", "w")

for i in range(N):
    xfile.write("%.5f\n" % x[i])
    yfile.write("%.5f\n" % y[i])
    zfile.write("%.5f\n" % z[i])

xfile.close()    
yfile.close()    
zfile.close()    

# Plot
plt.rcParams["figure.figsize"] = (10,10)
plt.rcParams.update({'font.size': 18})
ax = plt.figure().add_subplot(projection='3d')

ax.plot(x, y, z, lw=2)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_title("Lorenz Attractor")

plt.show()    

First we are going to look into how we define the generic equations, parameters, bounds etc. To do so, we will load $\texttt{File1.txt}$ and go over each line.

In [ ]:
%cat File1.txt

We see that...

In [ ]:
!python compile.py

Now that we have the data, we start by importing all the modules and functions needed to run our code.

In [ ]:
from header import *

import pyximport
pyximport.install()

from PyIpopt_Funks import *

import time

import pickle 

import os

from tqdm.notebook import tqdm_notebook

import OneLoopInC

from OneLoopInC import eval_f_tricky

from OneLoopInC import eval_grad_f_tricky

from OneLoopInC import eval_h_tricky

In [ ]:
# We load the variables needed from ObjNeed.obj created in Define_JacHess.py
file_ObjJacHess = open('ObjJacHess.obj', 'rb') 

ObjFunk_Meas_eval = pickle.load(file_ObjJacHess)
ObjFunk_Model_eval = pickle.load(file_ObjJacHess)
Jacobian_Meas = pickle.load(file_ObjJacHess)
Jacobian_Model = pickle.load(file_ObjJacHess)
Hessian_Meas = pickle.load(file_ObjJacHess)
Hessian_Model = pickle.load(file_ObjJacHess)
row_final = pickle.load(file_ObjJacHess)
col_final = pickle.load(file_ObjJacHess)
nnzh = pickle.load(file_ObjJacHess)

def eval_f(x):
    assert len(x) == num_total
    
    return eval_f_tricky(x,Rf)

def eval_grad_f(x):
    assert len(x) == num_total
    
    return eval_grad_f_tricky(x,Rf)
    
def eval_h(x,lagrange,obj_factor,flag):
    if flag:
        return (np.array(col_final),np.array(row_final))
    else:
        return eval_h_tricky(x,lagrange,obj_factor,flag,Rf)
        
class DAHSI():
    def objective(self, x):
        """Returns the scalar value of the objective given x."""
        return eval_f_tricky(x,Rf)

    def gradient(self, x):
        """Returns the gradient of the objective with respect to x."""
        return np.transpose(eval_grad_f_tricky(x,Rf))

    def constraints(self, x):
        """Returns the constraints."""
        return array([ ], float_)

    def jacobian(self, x):
        """Returns the Jacobian of the constraints with respect to x."""
        return np.array([])

    def hessianstructure(self):
        """Returns the row and column indices for non-zero vales of the
        Hessian."""

        # NOTE: The default hessian structure is of a lower triangular matrix,
        # therefore this function is redundant. It is included as an example
        # for structure callback.

#        return np.nonzero(np.tril(np.ones((num_total, num_total))))    
        return (np.array(col_final),np.array(row_final))

    def hessian(self, x, lagrange, obj_factor):
        """Returns the non-zero values of the Hessian."""        
        H = eval_h_tricky(x,lagrange,obj_factor,0,Rf)        

        row, col = self.hessianstructure()

        return H

In [ ]:
# Choose seed for random number generation.
IC = 0
np.random.seed(IC)

# Bounds for both state variables and parameters. 
x_L = np.ones((num_total))
x_U = np.ones((num_total))

for i in range(num_vars):
    x_L[i:-num_params:num_vars] = float(Input1[1+2*num_vars+num_params+i].split(",")[0])
    x_U[i:-num_params:num_vars] = float(Input1[1+2*num_vars+num_params+i].split(",")[1])
for i in range(num_params):
    x_L[-num_params+i] = float(Input1[1+3*num_vars+num_params+i].split(",")[0])
    x_U[-num_params+i] = float(Input1[1+3*num_vars+num_params+i].split(",")[1])

In [ ]:
file_name = "D%s_M%s_IC%s_Lorenz.dat" % (num_vars, num_meas, IC) 
file_results = os.path.join("outputfiles",file_name)
f = open(file_results,"w+")

lambd = lambd_0

x_jp = np.zeros((num_total))    

# Use appropriate initial conditions: for state variables, random; for parameters, set them all to 0.
x0 = (x_U-x_L)*np.random.rand(num_total)+x_L      
for i in range(num_meas):
    for k in range(0,num_vars*num_tpoints,num_vars):
        x0[k+i] = data[int(k/num_vars),i] 
for i in range(num_params):    
    x0[i-num_params] = 0
        
for i in range(num_total):
    x_jp[i] = x0[i]

lb = x_L
ub = x_U

cl = np.array([])
cu = np.array([])

nlp = cyipopt.Problem(
   n=num_total,
   m=0,
   problem_obj=DAHSI(),
   lb=lb,
   ub=ub,
   cl=cl,
   cu=cu,
)

# Change some options of the solver.
nlp.add_option('linear_solver', 'ma97')
#nlp.add_option('max_iter',100)
#nlp.add_option('tol',1.e-12)
nlp.add_option('mu_strategy', 'adaptive')
nlp.add_option('adaptive_mu_globalization', 'never-monotone-mode')
#nlp.add_option('bound_relax_factor', 0)
nlp.add_option('print_level',0)

In [ ]:
# Here starts the main loop
while lambd < lambd_max:   
    f = open(file_results,"a+")
	
    Rf0 = 1e-2

    for i in range(num_total):
        x_jp[i] = x0[i]    

    ax = plt.figure()
    for beta in tqdm_notebook(range(beta_max+1)):
        f = open(file_results,"a+")
        # Make note in results file which \lambda and \beta we are at.
        f.write("%f %f " % (lambd, beta))        

        # Controlling how much the model is enforced.
        Rf = Rf0*(alpha**beta)
  
        # Solve it via IPOPT (solution is x_jn).    
        x_jn, info = nlp.solve(x_jp)
            
        obj = info['obj_val']
        
        # We hard threshold the parameter part of the solution (the last num_params elements).
        for i in range(num_params):
    	    if abs(x_jn[i-num_params]) < lambd:
    	        x_jn[i-num_params] = 0

        # We set this solution as the initial condition for the next iteration of IPOPT. 
        x_jp = x_jn   

        # Write cost function value in file.
        f.write("%e " % obj)
        
        plt.semilogy(beta, obj, 'bo', markersize=10)
        
        #for k in range(num_total):
        #    f.write("%f " % x_jp[k])
        #f.write("\n")  

        for k in range(num_params):
    	    f.write("%f " % x_jp[k-num_params])
        f.write("\n")     
        
        f.close()

    # Increase \lambda value.       
    lambd = lambd+0.5
f.close()    

plt.rcParams["figure.figsize"] = (12,10)
plt.rcParams.update({'font.size': 18})

plt.xlabel("\\beta")
plt.ylabel("log(action)")
# ax.set_title("")

plt.grid(True)

plt.show()

In [ ]:
output_file = np.loadtxt('outputfiles/D3_M3_IC0_Lorenz.dat', unpack = False)

In [ ]:
output_file[-1,:]